In [ ]:
%load_ext autoreload
import deepdish as dd
import numpy as np
import pandas as pd
import json
from skimage import io
from notebook_utilities.display import stack_browser, display_array
from matplotlib import pyplot as plt
from fimpy.core.split_dataset import H5SplitDataset
%matplotlib notebook
from pathlib import Path
from scipy.ndimage import median_filter

In [ ]:
%autoreload
from fimpy.registration.reg_from_skimage import register_translation
from fimpy.registration.volume import (
    align_block_shift,
    find_shifts_sobel,
    shift_stack,
    sobel_stack)
from fimpy.registration.plane import align_single_planes_sobel, displacement_between_planes

# Check alignment function

In [ ]:
path = Path(r"J:\_Shared\experiments\E0032_luminance\v01_flashes\IO\181024_f1\181024_f1_io6fef05_6dpf_im\src")
ds = H5SplitDataset(path)

In [ ]:
i = 30
plane = dd.io.load(path / "src_{0:0>3}.h5".format(i), "/stack_4D")
display_array(plane[0,0,:,:])

In [ ]:
display_array(np.max(plane[:,0,:,:], 0))

### Sobel filtering of the plane

In [ ]:
prefilter_sigma = 4.3
ref = np.mean(ds[:10,:,:,:,], 0)
full_sob = sobel_stack(ref, prefilter_sigma)

In [ ]:
display_array(full_sob)

Calculate fft reference

In [ ]:
full_fft_reference = np.fft.fftn(sobel_stack(ref, prefilter_sigma))

In [ ]:
plane_sob = sobel_stack(plane[:,0,:,:].astype(np.float), prefilter_sigma)

In [ ]:
display_array(sobel_stack(plane[-1, :,:,:].astype(float), prefilter_sigma) - full_sob[i:i+1,:,:])

### Try registration

In [ ]:
find_shifts_sobel(plane[:1, :, :, :], np.fft.fftn(sobel_stack(ds[-1, i:i+1, :, :], prefilter_sigma)), prefilter_sigma)

In [ ]:
sample_im_f = np.fft.fftn(sobel_stack(plane[0, :,:,:], prefilter_sigma))
sample_im_l = np.fft.fftn(sobel_stack(plane[-1, :,:,:], prefilter_sigma))

In [ ]:
register_translation(sample_im_f, np.fft.fftn(full_sob[i:i+1,:,:]), space="fourier", upsample_factor=10, return_error=False)

In [ ]:
register_translation(sample_im_l, np.fft.fftn(full_sob[i:i+1,:,:]), upsample_factor=10, space="fourier", return_error=False)

In [ ]:
register_translation(sample_im_l, sample_im_f, upsample_factor=10, space="fourier", return_error=False)

### Test actual fimpy function

In [ ]:
find_shifts_sobel(plane[-2, :, :,:], np.fft.fftn(full_sob[i:i+1,:,:]), prefilter_sigma)

In [ ]:
align_single_planes

In [ ]:
def find_shifts_sobel_plane(plane, fft_ref, prefilter_sigma, window):
    

def _get_plane_shifts(ds, reference, plane_i, prefilter_sigma):
    find_shifts_sobel(ds[:,:,:], np.fft.fftn(full_sob[i+1:i+2,:,:]), prefilter_sigma)

In [ ]:
i = 30
stack = ds[:,j:j+1,:,:]
ref = np.mean(ds[:10,:,:,:], 0)

In [ ]:
find_shifts_sobel(stack[-1:, :, :, :], np.fft.fftn(sobel_stack(ref[i:i+1, :, :], prefilter_sigma)), prefilter_sigma)

In [ ]:
sob_ref = sobel_stack(ref, prefilter_sigma)

In [ ]:
a, shifts = align_single_planes_sobel(stack, np.fft.fftn(sob_ref[i:i+1, :, :]), 
                                      prefilter_sigma=prefilter_sigma, upsample_factor=10, maxshift=15)

In [ ]:
display_array(a)

### Alignment between planes

In [ ]:
display_array(ref)

In [ ]:
%autoreload
from fimpy.registration.plane import align_single_planes_sobel, displacement_between_planes, shift_planes
from scipy.ndimage.interpolation import shift

In [ ]:
shifts = displacement_between_planes(ref, upsampling=10)

In [ ]:
display_array(shift_planes(ref, shifts))

In [ ]:
meanvolume = ref # sobel_stack(ref, prefilter_sigma)
num_planes = meanvolume.shape[0]
centre_plane = int(num_planes // 2)

In [ ]:
aligned = np.empty_like(meanvolume)

In [ ]:
shifts = np.zeros((meanvolume.shape[0], 2))

for i in range(centre_plane, meanvolume.shape[0]-1):
    s, error, diffphase = register_translation(meanvolume[i,:,:], meanvolume[i+1,:,:], 10)
    shifts[i+1,:] = shifts[i,:] + s
for i in range(centre_plane, 0, -1):
    s, error, diffphase = register_translation(meanvolume[i,:,:], meanvolume[i-1,:,:], 10)
    shifts[i-1,:] = shifts[i,:] + s

In [ ]:
display_array(shift_planes(ds[:10,:,:,:], dict(shifts=shifts)))

## Combine all alignments for final function

In [ ]:
%load_ext autoreload

In [ ]:
% autoreload
from fimpy.registration.plane import align_single_planes_sobel, displacement_between_planes, shift_planes
from fimpy.registration.volume import sobel_stack

In [ ]:
path = Path(r"J:\_Shared\exp22_2p\EC\imported\180526 f1\src")
ds = H5SplitDataset(path)
ref = np.mean(ds[:10,:,:,:], 0)
prefilter_sigma = 3.3
upsampling=10

In [ ]:
sob_ref = sobel_stack(ref, prefilter_sigma)

In [ ]:
%%time
whole_stack = ds[:,:,:,:][::50, :,:,:]

In [ ]:
%%time

for i in whole_stack.shape[:, ]
shifts = align_single_planes_sobel(p[::10, :,:,:], np.fft.fftn(sob_ref[i:i+1, :, :]), 
                                   prefilter_sigma=prefilter_sigma, upsample_factor=10, maxshift=15)